
# 영화 추천 시스템 



# PROBLEM STATEMENT

- 추천시스템은 영화나 노래등을 추천하는데 사용되며, 주로 관심사나 이용 내역을 기반으로 추천한다.
- 이 노트북에서는, Item-based Collaborative Filtering 으로 추천시스템을 구현한다.

- Dataset MovieLens: https://grouplens.org/datasets/movielens/100k/ 


# STEP #0: LIBRARIES IMPORT


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

# STEP #1: IMPORT DATASET

### Movie_Id_Titles.csv 파일을 읽으세요.

In [4]:
movie_titles_df = pd.read_csv('Movie_Id_Titles.csv')

In [5]:
movie_titles_df

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


### 'u.data' 파일을 구글드라이브에서 열어보세요.

그러면, 탭으로 구분되어 있고, 맨 위에 컬럼이름이 없습니다.

따라서 컬럼이름을 'user_id', 'item_id', 'rating', 'timestamp' 로 지어주면서 데이터프레임으로 읽어오세요.

In [6]:
movies_rating_df = pd.read_csv('u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

In [7]:
movies_rating_df

                                    # timestamp => 몇년도부터 초로 센것

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742
...,...,...,...,...
99998,880,476,3,880175444
99999,716,204,5,879795543
100000,276,1090,1,874795795
100001,13,225,2,882399156


In [8]:
movie_titles_df.loc[movie_titles_df['item_id'] == 133 , ]

,item_id,title
132,133,Gone with the Wind (1939)


### timestamp 컬럼은 필요없으니, movies_rating_df 에서 아예 제거하시오.

In [9]:
movies_rating_df.head(3)

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949


In [10]:
movies_rating_df = movies_rating_df.drop(columns= 'timestamp')

In [11]:
movies_rating_df.head(2)

,user_id,item_id,rating
0,0,50,5
1,0,172,5


In [12]:
movie_titles_df.head(2)

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)


### 두개의 데이터프레임을 합치시오.

### 어떻게 합칠지 모양을 잘 보고 합칩니다.

In [13]:
pd.merge(movie_titles_df, movies_rating_df , on= 'item_id')

,item_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3
...,...,...,...,...
99998,1678,Mat' i syn (1997),863,1
99999,1679,B. Monkey (1998),863,3
100000,1680,Sliding Doors (1998),863,2
100001,1681,You So Crazy (1994),896,3


In [14]:
movies_rating_df = pd.merge(movie_titles_df, movies_rating_df , on= 'item_id')

In [15]:
# pd.merge(movies_rating_df , movies_titles_df , on= 'item_id')

In [16]:
pd.merge(movie_titles_df, movies_rating_df , on= 'item_id')

,item_id,title_x,title_y,user_id,rating
0,1,Toy Story (1995),Toy Story (1995),308,4
1,1,Toy Story (1995),Toy Story (1995),287,5
2,1,Toy Story (1995),Toy Story (1995),148,4
3,1,Toy Story (1995),Toy Story (1995),280,4
4,1,Toy Story (1995),Toy Story (1995),66,3
...,...,...,...,...,...
99998,1678,Mat' i syn (1997),Mat' i syn (1997),863,1
99999,1679,B. Monkey (1998),B. Monkey (1998),863,3
100000,1680,Sliding Doors (1998),Sliding Doors (1998),863,2
100001,1681,You So Crazy (1994),You So Crazy (1994),896,3


In [17]:
movies_rating_df = pd.merge(movie_titles_df, movies_rating_df , on= 'item_id')

# STEP #2: VISUALIZE DATASET

### 각 영화 제목별로, 별점에 대한 기본통계치(최대,최소,중앙,표준편차,1/4,3/4 값)를 보여주세요.

In [18]:
movies_rating_df

,item_id,title_x,title_y,user_id,rating
0,1,Toy Story (1995),Toy Story (1995),308,4
1,1,Toy Story (1995),Toy Story (1995),287,5
2,1,Toy Story (1995),Toy Story (1995),148,4
3,1,Toy Story (1995),Toy Story (1995),280,4
4,1,Toy Story (1995),Toy Story (1995),66,3
...,...,...,...,...,...
99998,1678,Mat' i syn (1997),Mat' i syn (1997),863,1
99999,1679,B. Monkey (1998),B. Monkey (1998),863,3
100000,1680,Sliding Doors (1998),Sliding Doors (1998),863,2
100001,1681,You So Crazy (1994),You So Crazy (1994),896,3


In [19]:
movies_rating_df.groupby('title')['rating'].describe()

KeyError: ignored

### 각 영화별 별점의 평균을 구하고, 이를 ratings_df_mean 에 저장하시오.

In [ ]:
ratings_df_mean = movies_rating_df.groupby('title')['rating'].mean()

### 각 영화별로, 몇개의 데이터가 있는지 구하고, 이를 ratings_df_count 에 저장하시오.

In [ ]:
movies_rating_df.head()

In [ ]:
movies_rating_df['title'].value_counts()

In [ ]:
ratings_df_count = movies_rating_df['title'].value_counts()

### 두 데이터프레임을 합치세요. 

In [ ]:
ratings_df_mean

In [ ]:
ratings_df_count.reset_index

In [ ]:
pd.DataFrame(data= [ratings_df_mean, ratings_df_count])

In [ ]:
ratings_mean_count_df =  

In [ ]:
ratings_df_mean

In [ ]:
ratings_df_mean.to_frame()

In [ ]:
ratings_df_mean1 = ratings_df_mean.to_frame()

In [ ]:
ratings_df_mean1.columns = ['mean']

In [ ]:
ratings_df_mean1

In [ ]:
ratings_df_count

In [ ]:
ratings_df_count.to_frame()

In [ ]:
ratings_df_count1 = ratings_df_count.to_frame()

In [ ]:
ratings_df_count1.columns = ['count']

In [ ]:
ratings_df_count1

In [ ]:
# 두개 데이터프레임 준비됐고 합치기

ratings_df_mean1.join(ratings_df_count1)

# pandas의 join 함수는, 데이터프레임끼리 합치는 함수이되, 
# 인덱스가 같은것들 끼리 알아서 합쳐준다

# 시리즈는 join 함수가 없습니다.

In [ ]:
ratings_mean_count_df =  ratings_df_mean1.join(ratings_df_count1)

### 컬럼명을 확인하면, 합쳐진 컬럼들이 rating, rating 이라고 되어있습니다. 이를 count, mean 으로 컬럼명을 셋팅하세요.

In [ ]:
ratings_mean_count_df

### mean 으로 히스토그램을 그려보세요.

In [ ]:
ratings_mean_count_df['mean'].hist()
plt.show()

### count 로 히스토그램을 그려보세요.

In [ ]:
ratings_mean_count_df['count'].hist()
plt.show()

### 평균점수가 5점인 영화들은 어떤 영화인지 확인하세요.

In [ ]:
ratings_mean_count_df.loc[ratings_mean_count_df['mean'] == 5 , ]

### count 가 가장 많은 것부터 정렬하여 100개까지만 보여주세요.

In [ ]:
ratings_mean_count_df.sort_values(by= 'count', ascending= False).head(100)

# STEP #3: 영화 하나에 대한, ITEM-BASED COLLABORATIVE FILTERING 수행!

In [ ]:
movie_titles_df.shape

In [ ]:
movies_rating_df.groupby('user_id')['user_id'].count()

# 표를 만들어야 된다는걸 외우자
# 위에 영화이름
# 옆에 유저아이디

In [ ]:
movies_rating_df.corr()

# 상관계수는 corr()을 써야되고
# corr()은 컬럼에 있어야됨

In [ ]:
movies_rating_df

In [ ]:
# movies_rating_df 를 가지고 아래 피봇테이블 합니다.

### 피봇 테이블을 하여, 콜라보레이티브 필터링 포맷으로 변경

In [ ]:
movies_rating_df.pivot_table(values= 'rating', index= 'user_id', columns='title', aggfunc= 'mean')

In [ ]:
df = movies_rating_df.pivot_table(values= 'rating', index= 'user_id', columns='title', aggfunc= 'mean')

### 전체 영화와, 타이타닉 영화의 상관관계 분석을 하면, 타이타닉을 본 사람들에게 상관계수가 높은 영화를 추천하면 된다. corrwith 함수를 이용한다.

In [ ]:
df['Titanic (1997)']

In [ ]:
titanic_corr = df.corrwith( df['Titanic (1997)'] )

In [ ]:
titanic_corr = titanic_corr.to_frame()

In [ ]:
titanic_corr

In [ ]:
titanic_corr.columns = ['correlation']

In [ ]:
titanic_corr

In [ ]:
titanic_corr = titanic_corr.join( ratings_mean_count_df['count'])

In [ ]:
titanic_corr.dropna(inplace = True)

In [ ]:
titanic_corr.sort_values('correlation', ascending= False)

In [ ]:
titanic_corr.loc[ titanic_corr['count'] > 80 , ]

In [ ]:
titanic_corr.loc[ titanic_corr['count'] > 80 , ].sort_values('correlation', ascending=False)

### 실습. star wars 를 본 사람들에게 영화를 추천할 것입니다. 5개의 추천 영화 제목을 찾으세요.

힌트 : 먼저 star wars 의 정확한 이름을 검색해서 찾으세요. 그리고 나서 스타워즈를 본 유저의 데이터를 가져와서, 위와 같이 상관관계분석을 합니다.

In [ ]:
df.columns.str.contains('star')

In [ ]:
df.columns.str.contains('star').shape

In [ ]:
df.shape

In [ ]:
df.columns.str.lower().contains('star')

In [ ]:
movie_titles_df['title'].str.lower().contains('star')

In [ ]:
movie_titles_df.loc[ movie_titles_df['title'].str.contains('star') , ]

In [ ]:

movie_titles_df['title'].values.str.lower().contains('star')

In [ ]:
movie_titles_df['title'].str.lower().contains('star')

In [ ]:
df.loc[ : , df.columns.str.lower().contains('star')]

In [ ]:
(df.columns.str.lower()).str.contains('star wars')

In [ ]:
df.loc[:,(df.columns.str.lower()).str.contains('star wars')]

In [ ]:
df.corrwith(df['Star Wars (1977)'])

In [ ]:
starwars_corr = df.corrwith(df['Star Wars (1977)'])

In [ ]:
starwars_corr = starwars_corr.to_frame()

In [ ]:
starwars_corr.columns = ['correlation']

In [ ]:
starwars_corr

In [ ]:
starwars_corr = starwars_corr.join( ratings_mean_count_df['count'] )

In [ ]:
starwars_corr

In [ ]:
starwars_corr.dropna(inplace=True)

In [ ]:
starwars_corr.sort_values('correlation', ascending=False)

In [ ]:
starwars_corr.loc[ starwars_corr['count'] > 80 , ].sort_values('correlation' , ascending = False).head(6)

# STEP#4: 전체 데이터셋에 대한 ITEM-BASED COLLABORATIVE FILTER 를 만들자!

In [ ]:
df

In [ ]:
# 최소 80개이상 데이터가 있는것만 상관계수 뽑아라
df.corr(min_periods= 80)

In [ ]:
movie_corr = df.corr(min_periods= 80)

### 나의 별점 정보를 가지고, 영화를 추천해 달라고 할 것이다! 

내 영화 별점 정보는 My_Ratings.csv 파일에 있다. 

In [ ]:
myRatings = pd.read_csv('My_Ratings.csv')
 

In [ ]:
myRatings

# 코드를 짤땐 한번에할라 하지말고 하나씩

In [ ]:
movie_name = myRatings['Movie Name'][0]

In [ ]:
movie_name

In [ ]:
movie_corr

In [ ]:
movie_corr[movie_name]

In [ ]:
movie_corr[movie_name].dropna()

In [ ]:
movie_corr[movie_name].dropna().sort_values(ascending= False)

In [ ]:
recom_movies = movie_corr[movie_name].dropna().sort_values(ascending= False)

In [ ]:
recom_movies = movie_corr[movie_name].dropna().sort_values(ascending= False).to_frame()

In [ ]:
recom_movies

In [ ]:
recom_movies.columns = ['correlation']

In [ ]:
recom_movies

In [ ]:
myRatings

In [ ]:
myRatings['Ratings'][0]

In [ ]:
recom_movies['W'] = myRatings['Ratings'][0] * recom_movies['correlation']

## 이제는 나머지 내가 본 영화들도 추천영화 똑같이 가져오자.

In [ ]:
myRatings

## 위의 추천영화 작업을 자동화 하기 위한 파이프라인을 만드시오. 

힌트 : 반복문을 사용하여 비슷한영화에 대한 데이터프레임을 만들고, 이를 아래 빈 데이터프레임에 계속하여 추가하시오. 반복문이 끝나면, 아래 데이터프레임을 wegiht 컬럼으로 정렬하면 됩니다.

In [ ]:
similar_movies_list = pd.DataFrame()